### Dataset: Amazon Music
### Algorithm: RecentPop
### Padding unknown user/item: N.A.
### Flow: Library Flow

In [1]:
k = 100
from streamsight.datasets.amazon import AmazonMusicDataset
from streamsight.preprocessing import MinItemsPerUser
from streamsight.settings import SlidingWindowSetting
dataset = AmazonMusicDataset()
dataset.add_filter(
    MinItemsPerUser(3, AmazonMusicDataset.ITEM_IX, AmazonMusicDataset.USER_IX)
)
data = dataset.load()

setting_window = SlidingWindowSetting(
    background_t=1546241746,
    window_size=60 * 60 * 24 * 365, # 365 days
    top_K=k
)

setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - AmazonMusicDataset being initialized with 'data' as the base path.
DEBUG - AmazonMusicDataset is initialized.
INFO - AmazonMusicDataset is loading dataset...
DEBUG - Data file is in memory and in dir specified.
DEBUG - AmazonMusicDataset applying filters set.
DEBUG - 	interactions before preprocess: 130434
DEBUG - 	items before preprocess: 70511
DEBUG - 	users before preprocess: 100952
DEBUG - applying filter: MinItemsPerUser(min_iu=3, count_duplicates=True, item_ix=parent_asin, user_ix=user_id)
DEBUG - 	interactions after filter: 25542
DEBUG - 	items after filter: 20836
DEBUG - 	users after filter: 4464
DEBUG - 	interactions after preprocess: 25542
DEBUG - 

  0%|          | 0/4 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=1546241746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1546241746
DEBUG - Performing lt(t, 1577777746)
DEBUG - Performing ge(t, 1546241746)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1546241746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1546241746 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=1546241746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1577777746
DEBUG - Performing lt(t, 1609313746)
DEBUG - Performing ge(t, 1577777746)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1577777746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1577777746 re

5it [00:00, 94.87it/s]               

INFO - Finished split with window size 31536000 seconds. Number of splits: 5 in total.
INFO - SlidingWindowSetting data split - Took 0.0781s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [2]:
from streamsight.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
builder.add_metric("NDCGK")
evaluator = builder.build()

In [3]:
from streamsight.algorithms import RecentPopularity
external_model = RecentPopularity(K=k)
external_model_id = evaluator.register_algorithm(algorithm_name="AmazonMusic_RecentPop")
print(external_model_id)

INFO - Registering algorithm name AmazonMusic_RecentPop with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [4]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (3733, 16060)
DEBUG - Shape of dataframe stored in matrix was (19253, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (19253, 6) after masking
DEBUG - Final (user x item) shape defined is (3733, 16060)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (3733, 16060)
DEBUG - Shape of dataframe stored in matrix was (1719, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1719, 6) after masking
DEBUG - Final (user x item) shape defined is (3733, 16060)
DEBUG - (user x item) shape defined is (3733, 16060)
DEBUG - Shape of dataframe stored in matrix was (1719, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (295, 6) after masking
DEBUG - Final (user x item) shape defined is (3957, 16060)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [5]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
Sorted scores recentpop:  [0.95652174 0.         0.         ... 0.         0.         0.        ]
DEBUG - Fitting RecentPopularity complete - Took 0.00235s


       interactionid   uid    iid          ts  rating  helpful_vote
0                  0     0      0   906574660     5.0             1
1                  1     0      1   908892059     3.0             0
2                  2     1      2   920935758     5.0             0
3                  3     2      3   941546039     4.0             0
4                  4     1      4   942757962     5.0            20
...              ...   ...    ...         ...     ...           ...
19248          19248  3730  15828  1546039809     5.0             1
19249          19249  3730  16057  1546039948     5.0             0
19250          19250  3731  16058  1546042248     4.0             0
19251          19251  1620  16059  1546057175     5.0             2
19252          19252  3732  14415  1546129819     3.0             1

[19253 rows x 6 columns]

In [6]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [3732, 3733, 1561, 1787, 3734, 38, 1600, 2594, 289, 1788, 2179, 3735, 3736, 3583, 3720, 3445, 3570, 2584, 1659, 3737, 3738, 1948, 3690, 827, 2191, 2428, 2915, 3739, 2510, 3641, 3730, 879, 3031, 3703, 271, 2652, 3740, 3062, 581, 3741, 1265, 2756, 856, 3244, 640, 3648, 3716, 1858, 101, 3742, 241, 3424, 3743, 1241, 83, 991, 3744, 3228, 1042, 1083, 3196, 1587, 3745, 3605, 3746, 1361, 3260, 2429, 785, 3688, 2887, 3639, 3747, 3748, 3692, 3478, 3749, 2279, 3750, 1982, 1727, 586, 3581, 3751, 2765, 2307, 1284, 1194, 2254, 965, 3752, 3715, 1127, 1802, 3597, 2821, 2295, 3753, 3754, 2309, 3157, 2282, 3755, 1001, 1052, 3756, 2365, 3598, 3008, 859, 3757, 1669, 1640, 176, 3758, 1004, 3759, 3622, 2602, 3238, 3064, 3760, 3761, 1362, 191, 3211, 1466, 923, 3762, 3763, 3710, 2783, 718, 519, 2134, 2113, 3602, 3458, 3433, 2301, 221, 3764, 1075, 3765, 2354, 1341, 512, 155, 2345, 77

       interactionid   uid  iid          ts  rating  helpful_vote
19253          19253  3732   -1  1546243901     3.0             0
19254          19254  3733   -1  1546295208     5.0             0
19255          19255  1561   -1  1546307829     5.0             7
19256          19256  1787   -1  1546308024     1.0             0
19257          19257  1787   -1  1546308362     1.0             0
...              ...   ...  ...         ...     ...           ...
20967          20967   212   -1  1577729682     5.0             0
20968          20968   522   -1  1577753078     5.0             0
20969          20969  3076   -1  1577754015     5.0             0
20970          20970   753   -1  1577754537     5.0             0
20971          20971  3288   -1  1577755529     5.0             0

[1719 rows x 6 columns]

In [7]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 81900 stored elements and shape (3957, 16060)>
  Coords	Values
  (3, 0)	0.9565217391304348
  (3, 9)	1.0
  (3, 46)	0.2608695652173913
  (3, 49)	0.9130434782608695
  (3, 51)	0.21739130434782608
  (3, 54)	0.21739130434782608
  (3, 251)	0.21739130434782608
  (3, 264)	0.30434782608695654
  (3, 314)	0.391304347826087
  (3, 703)	0.21739130434782608
  (3, 750)	0.2608695652173913
  (3, 752)	0.21739130434782608
  (3, 837)	0.2608695652173913
  (3, 1116)	0.2608695652173913
  (3, 1262)	0.21739130434782608
  (3, 1379)	0.21739130434782608
  (3, 1391)	0.2608695652173913
  (3, 1408)	0.391304347826087
  (3, 1521)	0.21739130434782608
  (3, 1539)	0.34782608695652173
  (3, 1581)	0.21739130434782608
  (3, 1586)	0.2608695652173913
  (3, 1587)	0.34782608695652173
  (3, 2019)	0.34782608695652173
  (3, 2175)	0.21739130434782608
  :	:
  (3956, 7978)	0.21739130434782608
  (3956, 8159)	0.21739130434782608
  (3956, 8238)	0.34782608695652173
  (3956, 8268

# Iteration 2

In [8]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (3957, 17393)
DEBUG - Shape of dataframe stored in matrix was (1719, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1719, 6) after masking
DEBUG - Final (user x item) shape defined is (3957, 17393)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (3957, 17393)
DEBUG - Shape of dataframe stored in matrix was (1267, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1267, 6) after masking
DEBUG - Final (user x item) shape defined is (3957, 17393)
DEBUG - (user x item) shape defined is (3957, 17393)
DEBUG - Shape of dataframe stored in matrix was (1267, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (234, 6) after masking
DEBUG - Final (user x item) shape defined is (4109, 17393)
DEBUG - Data cached for step 2 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fitting 

       interactionid   uid    iid          ts  rating  helpful_vote
19253          19253  3732  16060  1546243901     3.0             0
19254          19254  3733  16061  1546295208     5.0             0
19255          19255  1561  16062  1546307829     5.0             7
19256          19256  1787  16063  1546308024     1.0             0
19257          19257  1787  16064  1546308362     1.0             0
...              ...   ...    ...         ...     ...           ...
20967          20967   212   7561  1577729682     5.0             0
20968          20968   522  17389  1577753078     5.0             0
20969          20969  3076  17390  1577754015     5.0             0
20970          20970   753  17391  1577754537     5.0             0
20971          20971  3288  17392  1577755529     5.0             0

[1719 rows x 6 columns]

In [9]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [753, 3551, 3957, 791, 1447, 3202, 3958, 3959, 2785, 3703, 2086, 3830, 3944, 2239, 3184, 3727, 189, 1124, 3960, 3961, 3810, 3773, 3962, 543, 812, 3211, 2362, 2405, 3859, 990, 3823, 3951, 1265, 646, 2169, 2390, 2894, 3963, 3964, 3200, 3965, 3966, 1271, 3914, 3967, 1754, 1902, 3968, 1258, 3969, 2807, 1982, 830, 2618, 3840, 840, 3816, 3795, 1060, 3955, 3970, 3496, 3971, 2295, 3755, 2186, 1451, 3972, 2550, 281, 1730, 1536, 3807, 3734, 3766, 3973, 1042, 2886, 2406, 3841, 3563, 2827, 1684, 904, 517, 1391, 3501, 3790, 2417, 3912, 1194, 784, 3974, 2260, 3975, 586, 3712, 3870, 899, 719, 3976, 3772, 3163, 2716, 2318, 2489, 3977, 3761, 2199, 563, 3978, 3661, 3472, 3333, 1550, 3950, 3033, 1144, 130, 3979, 3800, 972, 3073, 3895, 3447, 3952, 3980, 1201, 2748, 3721, 1276, 1498, 3522, 3981, 3359, 883, 76, 3982, 3595, 3983, 399, 3405, 1736, 3746, 980, 3754, 3984, 463, 2216, 1

       interactionid   uid  iid          ts  rating  helpful_vote
20972          20972   753   -1  1577895301     5.0             5
20973          20973  3551   -1  1577900463     5.0             0
20974          20974  3551   -1  1577900694     4.0             0
20975          20975  3957   -1  1577912436     5.0             0
20976          20976  3957   -1  1577912562     5.0             2
...              ...   ...  ...         ...     ...           ...
22234          22234  4077   -1  1609103063     5.0             0
22235          22235  4058   -1  1609183165     5.0             1
22236          22236  1265   -1  1609209509     5.0             2
22237          22237  2296   -1  1609217665     5.0             0
22238          22238  4102   -1  1609263547     5.0             0

[1267 rows x 6 columns]

In [10]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 63500 stored elements and shape (4111, 17393)>
  Coords	Values
  (3, 235)	0.2857142857142857
  (3, 4817)	0.2857142857142857
  (3, 5860)	0.42857142857142855
  (3, 6450)	0.2857142857142857
  (3, 6579)	0.2857142857142857
  (3, 10065)	0.2857142857142857
  (3, 10080)	0.2857142857142857
  (3, 11670)	0.2857142857142857
  (3, 11965)	0.2857142857142857
  (3, 12504)	0.2857142857142857
  (3, 12803)	0.2857142857142857
  (3, 13112)	0.2857142857142857
  (3, 13486)	0.2857142857142857
  (3, 14052)	0.2857142857142857
  (3, 14345)	0.2857142857142857
  (3, 14707)	0.5714285714285714
  (3, 14755)	0.42857142857142855
  (3, 15026)	0.42857142857142855
  (3, 15108)	0.2857142857142857
  (3, 15492)	0.2857142857142857
  (3, 15542)	0.2857142857142857
  (3, 15589)	0.2857142857142857
  (3, 15593)	0.42857142857142855
  (3, 15602)	0.2857142857142857
  (3, 15603)	0.2857142857142857
  :	:
  (4110, 16424)	0.2857142857142857
  (4110, 16440)	0.2857142857142857
 

# Iteration 3

In [11]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4111, 18338)
DEBUG - Shape of dataframe stored in matrix was (1267, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1267, 6) after masking
DEBUG - Final (user x item) shape defined is (4111, 18338)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (4111, 18338)
DEBUG - Shape of dataframe stored in matrix was (1597, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1597, 6) after masking
DEBUG - Final (user x item) shape defined is (4111, 18338)
DEBUG - (user x item) shape defined is (4111, 18338)
DEBUG - Shape of dataframe stored in matrix was (1597, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (254, 6) after masking
DEBUG - Final (user x item) shape defined is (4309, 18338)
DEBUG - Data cached for step 3 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fitting 

       interactionid   uid    iid          ts  rating  helpful_vote
20972          20972   753  17393  1577895301     5.0             5
20973          20973  3551  17394  1577900463     5.0             0
20974          20974  3551  17395  1577900694     4.0             0
20975          20975  3957  17396  1577912436     5.0             0
20976          20976  3957  17397  1577912562     5.0             2
...              ...   ...    ...         ...     ...           ...
22234          22234  4077  14828  1609103063     5.0             0
22235          22235  4058  18335  1609183165     5.0             1
22236          22236  1265  18336  1609209509     5.0             2
22237          22237  2296  18337  1609217665     5.0             0
22238          22238  4102  17660  1609263547     5.0             0

[1267 rows x 6 columns]

In [12]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [791, 2342, 3633, 1127, 2673, 4091, 3918, 4111, 4112, 2186, 2730, 649, 3685, 3761, 1130, 4113, 3856, 3258, 3208, 4026, 4002, 603, 812, 1160, 3103, 4114, 4097, 3132, 4115, 3966, 3179, 2252, 4076, 1265, 2643, 3853, 753, 2069, 3830, 2392, 4047, 4116, 3929, 4117, 1962, 1223, 392, 4105, 543, 3521, 4077, 1767, 4095, 2359, 378, 1042, 4093, 829, 3597, 4025, 4118, 4119, 3734, 773, 3434, 2488, 3002, 4099, 3476, 4098, 4120, 4121, 281, 4000, 1615, 4122, 140, 4060, 2645, 4123, 4124, 3973, 3933, 3623, 2870, 4125, 4001, 3557, 2337, 4126, 968, 845, 998, 3413, 4127, 4128, 4129, 2549, 4023, 814, 2047, 2664, 2972, 349, 1939, 3823, 489, 4130, 2135, 4131, 3449, 3601, 529, 3158, 4132, 4016, 3657, 3316, 3888, 3472, 2886, 3921, 2198, 4133, 3293, 500, 3850, 4027, 1233, 3722, 4134, 4135, 1901, 3653, 1336, 4136, 4088, 3647, 4137, 2893, 4138, 4015, 4139, 4140, 4013, 4075, 3957, 991, 382

       interactionid   uid  iid          ts  rating  helpful_vote
22239          22239   791   -1  1609352713     2.0             0
22240          22240  2342   -1  1609352784     5.0             1
22241          22241   791   -1  1609352935     1.0             0
22242          22242  3633   -1  1609429831     5.0             0
22243          22243  1127   -1  1609435015     4.0             0
...              ...   ...  ...         ...     ...           ...
23831          23831  4307   -1  1640701893     5.0             0
23832          23832  4307   -1  1640702119     5.0             0
23833          23833  4307   -1  1640703766     5.0             1
23834          23834  4308   -1  1640709415     5.0             2
23835          23835  3959   -1  1640710325     5.0             1

[1597 rows x 6 columns]

In [13]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 73800 stored elements and shape (4309, 18338)>
  Coords	Values
  (22, 10124)	0.4
  (22, 10306)	0.4
  (22, 12537)	0.4
  (22, 12600)	0.4
  (22, 12666)	0.8
  (22, 13870)	0.4
  (22, 14277)	0.4
  (22, 14707)	0.4
  (22, 14828)	0.4
  (22, 15554)	0.6
  (22, 15777)	1.0
  (22, 15796)	0.4
  (22, 15804)	0.4
  (22, 15845)	0.4
  (22, 15962)	1.0
  (22, 16205)	0.4
  (22, 16527)	0.4
  (22, 16571)	0.4
  (22, 16814)	0.8
  (22, 16975)	0.4
  (22, 17062)	0.4
  (22, 17091)	0.8
  (22, 17223)	0.4
  (22, 17398)	0.4
  (22, 17416)	0.4
  :	:
  (4308, 17669)	0.4
  (4308, 17683)	0.6
  (4308, 17687)	0.4
  (4308, 17723)	0.4
  (4308, 17734)	0.4
  (4308, 17740)	0.8
  (4308, 17743)	0.6
  (4308, 17806)	0.4
  (4308, 17819)	0.4
  (4308, 17826)	0.4
  (4308, 17845)	0.4
  (4308, 17848)	0.6
  (4308, 17875)	0.4
  (4308, 17913)	0.4
  (4308, 17923)	0.4
  (4308, 17929)	0.4
  (4308, 17932)	0.4
  (4308, 17961)	0.4
  (4308, 18014)	0.4
  (4308, 18016)	0.4
  (4308, 18017)	0.4

# Iteration 4

In [14]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4309, 19548)
DEBUG - Shape of dataframe stored in matrix was (1597, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1597, 6) after masking
DEBUG - Final (user x item) shape defined is (4309, 19548)
DEBUG - Caching evaluation data for step 4
DEBUG - (user x item) shape defined is (4309, 19548)
DEBUG - Shape of dataframe stored in matrix was (1259, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1259, 6) after masking
DEBUG - Final (user x item) shape defined is (4309, 19548)
DEBUG - (user x item) shape defined is (4309, 19548)
DEBUG - Shape of dataframe stored in matrix was (1259, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (197, 6) after masking
DEBUG - Final (user x item) shape defined is (4424, 19548)
DEBUG - Data cached for step 4 complete
Sorted scores recentpop:  [0.4 0.  0.  ... 0.  0.  0. ]
DEBUG - Fi

       interactionid   uid    iid          ts  rating  helpful_vote
22239          22239   791  18338  1609352713     2.0             0
22240          22240  2342  18339  1609352784     5.0             1
22241          22241   791  18340  1609352935     1.0             0
22242          22242  3633  18341  1609429831     5.0             0
22243          22243  1127  18342  1609435015     4.0             0
...              ...   ...    ...         ...     ...           ...
23831          23831  4307  19547  1640701893     5.0             0
23832          23832  4307  11670  1640702119     5.0             0
23833          23833  4307  12786  1640703766     5.0             1
23834          23834  4308  12399  1640709415     5.0             2
23835          23835  3959  18308  1640710325     5.0             1

[1597 rows x 6 columns]

In [15]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [4230, 1548, 1677, 1680, 3685, 3293, 4309, 4304, 753, 4065, 2261, 3859, 4150, 681, 791, 4310, 4287, 2302, 1898, 4292, 3681, 4311, 2854, 130, 4208, 241, 2905, 3864, 3296, 1103, 268, 4048, 4223, 4281, 4009, 2615, 2637, 4303, 4135, 4312, 4313, 4314, 4278, 169, 3693, 3158, 4315, 4021, 4216, 4316, 19, 2129, 3052, 4317, 1033, 4264, 4109, 3953, 4213, 4155, 2445, 4318, 4147, 1570, 3267, 2277, 917, 4319, 543, 4183, 943, 4320, 711, 4321, 1447, 4238, 4255, 3513, 2332, 4200, 2829, 3925, 4245, 1265, 4172, 4322, 4130, 4291, 4323, 4212, 942, 2886, 4324, 4325, 53, 3841, 4326, 4282, 3706, 4088, 3664, 830, 3577, 3214, 3968, 2673, 4047, 378, 4293, 4162, 2832, 3636, 696, 3584, 3984, 140, 1422, 1446, 3389, 57, 2409, 4144, 1750, 4042, 3428, 3110, 1975, 3297, 1052, 3905, 3245, 4327, 4328, 3853, 3635, 3962, 4329, 4073, 3915, 121, 2650, 1844, 4074, 4301, 4022, 4201, 3875, 2187, 1739,

       interactionid   uid  iid          ts  rating  helpful_vote
23836          23836  4230   -1  1640869031     5.0             1
23837          23837  1548   -1  1640883197     5.0             1
23838          23838  1677   -1  1640884401     5.0             4
23839          23839  1677   -1  1640885026     5.0             0
23840          23840  1680   -1  1640885425     3.0             1
...              ...   ...  ...         ...     ...           ...
25090          25090  2489   -1  1672339062     5.0             0
25091          25091  4129   -1  1672362059     5.0             0
25092          25092  4129   -1  1672362059     5.0             0
25093          25093  3250   -1  1672370323     5.0             0
25094          25094  4176   -1  1672374881     5.0             0

[1259 rows x 6 columns]

In [16]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 59700 stored elements and shape (4431, 19548)>
  Coords	Values
  (19, 0)	0.4
  (19, 1291)	0.4
  (19, 6188)	0.4
  (19, 11670)	0.4
  (19, 12163)	0.4
  (19, 14320)	0.6
  (19, 14707)	0.6
  (19, 15260)	0.4
  (19, 15777)	0.8
  (19, 15925)	0.6
  (19, 15962)	0.8
  (19, 16091)	1.0
  (19, 17091)	0.4
  (19, 17431)	0.8
  (19, 17513)	0.4
  (19, 17683)	0.4
  (19, 17699)	0.8
  (19, 17743)	0.6
  (19, 17848)	0.4
  (19, 17902)	0.4
  (19, 17990)	0.6
  (19, 18225)	0.4
  (19, 18272)	0.4
  (19, 18338)	0.4
  (19, 18357)	0.4
  :	:
  (4430, 18842)	0.4
  (4430, 18871)	0.4
  (4430, 18920)	0.4
  (4430, 18924)	0.4
  (4430, 18944)	0.4
  (4430, 18959)	0.6
  (4430, 18972)	0.4
  (4430, 19001)	0.4
  (4430, 19023)	0.4
  (4430, 19038)	0.4
  (4430, 19044)	0.4
  (4430, 19085)	0.4
  (4430, 19092)	0.4
  (4430, 19117)	0.4
  (4430, 19118)	0.4
  (4430, 19121)	0.4
  (4430, 19143)	0.6
  (4430, 19173)	0.4
  (4430, 19177)	0.4
  (4430, 19185)	0.4
  (4430, 19187)	0.4
  (44

# Iteration 5

In [17]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4431, 20527)
DEBUG - Shape of dataframe stored in matrix was (1259, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1259, 6) after masking
DEBUG - Final (user x item) shape defined is (4431, 20527)
DEBUG - Caching evaluation data for step 5
DEBUG - (user x item) shape defined is (4431, 20527)
DEBUG - Shape of dataframe stored in matrix was (447, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (447, 6) after masking
DEBUG - Final (user x item) shape defined is (4431, 20527)
DEBUG - (user x item) shape defined is (4431, 20527)
DEBUG - Shape of dataframe stored in matrix was (447, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (105, 6) after masking
DEBUG - Final (user x item) shape defined is (4462, 20527)
DEBUG - Data cached for step 5 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fitting Rec

       interactionid   uid    iid          ts  rating  helpful_vote
23836          23836  4230  18308  1640869031     5.0             1
23837          23837  1548  18862  1640883197     5.0             1
23838          23838  1677  19548  1640884401     5.0             4
23839          23839  1677  17482  1640885026     5.0             0
23840          23840  1680  19549  1640885425     3.0             1
...              ...   ...    ...         ...     ...           ...
25090          25090  2489  20523  1672339062     5.0             0
25091          25091  4129  20524  1672362059     5.0             0
25092          25092  4129  20524  1672362059     5.0             0
25093          25093  3250  20525  1672370323     5.0             0
25094          25094  4176  20526  1672374881     5.0             0

[1259 rows x 6 columns]

In [18]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [4339, 4279, 1466, 970, 4308, 4349, 1777, 3389, 4411, 4431, 2716, 4373, 4154, 4083, 4088, 4418, 928, 543, 3242, 3672, 1736, 4405, 3973, 4428, 1426, 4432, 4433, 4071, 4356, 1505, 4280, 3827, 4434, 4425, 4104, 3472, 4176, 243, 4435, 3179, 4429, 3868, 2770, 4436, 4304, 4398, 4391, 4415, 2224, 4364, 4365, 4424, 811, 3644, 4087, 2655, 4437, 4206, 711, 4223, 4288, 4325, 378, 4331, 4045, 4030, 4348, 1625, 828, 221, 4330, 1127, 4438, 1042, 4439, 3927, 3977, 4440, 917, 4441, 4053, 2981, 753, 1504, 2178, 4413, 4258, 4386, 4442, 4443, 965, 4286, 2954, 4194, 4444, 3196, 2774, 3756, 4417, 1743, 110, 2755, 2389, 78, 1135, 393, 4262, 3297, 4052, 1975, 3217, 4384, 3089, 4426, 2688, 4067, 942, 1637, 850, 4123, 4298, 3667, 4220, 4445, 2853, 3759, 2454, 1880, 4399, 4446, 4081, 1590, 3843, 1548, 3543, 22, 1419, 4293, 4335, 4403, 2673, 3528, 1039, 4338, 2459, 4205, 3505, 3115, 22

       interactionid   uid  iid          ts  rating  helpful_vote
25095          25095  4339   -1  1672406677     5.0             0
25096          25096  4339   -1  1672406685     5.0             0
25097          25097  4279   -1  1672427352     5.0             0
25098          25098  4279   -1  1672427477     5.0             0
25099          25099  1466   -1  1672440344     5.0             0
...              ...   ...  ...         ...     ...           ...
25537          25537  4462   -1  1691639002     5.0             0
25538          25538  4463   -1  1692200640     5.0             0
25539          25539  4463   -1  1692202028     5.0             0
25540          25540  4463   -1  1692202101     5.0             0
25541          25541  2896   -1  1692267495     5.0             0

[447 rows x 6 columns]

In [19]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 24100 stored elements and shape (4464, 20527)>
  Coords	Values
  (22, 12163)	0.5
  (22, 15882)	0.5
  (22, 17482)	0.5
  (22, 17613)	0.5
  (22, 17848)	0.5
  (22, 17896)	0.75
  (22, 18285)	0.5
  (22, 18442)	0.5
  (22, 18573)	0.75
  (22, 18773)	0.75
  (22, 18907)	0.5
  (22, 18944)	0.5
  (22, 18955)	0.5
  (22, 18987)	1.0
  (22, 19016)	0.5
  (22, 19084)	0.5
  (22, 19091)	0.5
  (22, 19467)	0.5
  (22, 19556)	0.75
  (22, 19591)	0.5
  (22, 19594)	0.5
  (22, 19638)	0.5
  (22, 19646)	0.5
  (22, 19649)	0.5
  (22, 19656)	0.5
  :	:
  (4463, 19917)	0.5
  (4463, 19935)	0.5
  (4463, 19965)	0.5
  (4463, 19968)	0.5
  (4463, 19978)	0.5
  (4463, 19988)	0.5
  (4463, 20002)	0.5
  (4463, 20016)	0.5
  (4463, 20018)	0.5
  (4463, 20047)	0.5
  (4463, 20057)	0.75
  (4463, 20086)	0.5
  (4463, 20105)	0.5
  (4463, 20177)	0.5
  (4463, 20247)	0.5
  (4463, 20265)	0.5
  (4463, 20286)	0.75
  (4463, 20298)	0.5
  (4463, 20309)	0.5
  (4463, 20327)	0.5
  (4463, 2036

# evaluate metrics

In [20]:
evaluator.get_all_algorithm_status()

{'AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [21]:
evaluator.metric_results("macro")

macro_score  \
Algorithm                                          Metric                        
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... NDCGK_100          0.038817   
                                                   PrecisionK_100     0.001903   
                                                   RecallK_100        0.137740   

                                                                   num_window  
Algorithm                                          Metric                      
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... NDCGK_100                5  
                                                   PrecisionK_100           5  
                                                   RecallK_100              5

In [22]:
evaluator.metric_results("micro")


micro_score  \
Algorithm                                          Metric                       
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... NDCGK_100         0.035064   
                                                   PrecisionK_100    0.001708   
                                                   RecallK_100       0.120226   

                                                                   num_user  
Algorithm                                          Metric                    
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... NDCGK_100            761  
                                                   PrecisionK_100       761  
                                                   RecallK_100          761

In [23]:
evaluator.metric_results("user")

user_id  \
Algorithm                                          Timestamp    Metric                   
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... t=1546241746 PrecisionK_100     512   
                                                                PrecisionK_100    1024   
                                                                PrecisionK_100    3587   
                                                                PrecisionK_100    3590   
                                                                PrecisionK_100    3079   
...                                                                                ...   
                                                   t=1672385746 NDCGK_100         3956   
                                                                NDCGK_100         4348   
                                                                NDCGK_100         4220   
                                                                NDCGK_100         4349   
                                                                NDCGK_100         4223   

                                                                                   score  
Algorithm                                          Timestamp    Metric                    
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... t=1546241746 PrecisionK_100       0.0  
                                                                PrecisionK_100       0.0  
                                                                PrecisionK_100      0.01  
                                                                PrecisionK_100       0.0  
                                                                PrecisionK_100       0.0  
...                                                                                  ...  
                                                   t=1672385746 NDCGK_100       0.201849  
                                                                NDCGK_100            0.0  
                                                                NDCGK_100            0.0  
                                                                NDCGK_100            0.0  
                                                                NDCGK_100            0.0  

[2283 rows x 2 columns]

In [24]:
evaluator.metric_results("window")

window_score  \
Algorithm                                          Timestamp    Metric                         
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... t=1546241746 PrecisionK_100      0.001045   
                                                                RecallK_100         0.059287   
                                                                NDCGK_100           0.013621   
                                                   t=1577777746 PrecisionK_100      0.001890   
                                                                RecallK_100         0.130285   
                                                                NDCGK_100           0.036378   
                                                   t=1609313746 PrecisionK_100      0.001742   
                                                                RecallK_100         0.119850   
                                                                NDCGK_100           0.050729   
                                                   t=1640849746 PrecisionK_100      0.001644   
                                                                RecallK_100         0.129281   
                                                                NDCGK_100           0.030116   
                                                   t=1672385746 PrecisionK_100      0.003194   
                                                                RecallK_100         0.250000   
                                                                NDCGK_100           0.063242   

                                                                                num_user  
Algorithm                                          Timestamp    Metric                    
AmazonMusic_RecentPop_bdd640fb-0667-4ad1-9c80-3... t=1546241746 PrecisionK_100       201  
                                                                RecallK_100          201  
                                                                NDCGK_100            201  
                                                   t=1577777746 PrecisionK_100       164  
                                                                RecallK_100          164  
                                                                NDCGK_100            164  
                                                   t=1609313746 PrecisionK_100       178  
                                                                RecallK_100          178  
                                                                NDCGK_100            178  
                                                   t=1640849746 PrecisionK_100       146  
                                                                RecallK_100          146  
                                                                NDCGK_100            146  
                                                   t=1672385746 PrecisionK_100        72  
                                                                RecallK_100           72  
                                                                NDCGK_100             72